In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import logit

from IPython.display import display

from keras.layers import (Input, Dense, Lambda, Flatten, Reshape, BatchNormalization, Layer,
                          Activation, Dropout, Conv2D, Conv2DTranspose,
                          Concatenate, Add, Multiply)
from keras.engine import InputSpec
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import RMSprop, Adam
from keras.models import Model
from keras import metrics
from keras import backend as K
from keras_tqdm import TQDMNotebookCallback
from keras.datasets import cifar10


%matplotlib inline

In [112]:
class Mask(Layer):
    def __init__(self, mask_type, *args, **kwargs):
        mask_type in ['check_even', 'check_odd', 'channel_even', 'channel_odd']
        self.mask_type = mask_type
        super().__init__(*args, **kwargs)

    def build_mask(self, input_shape):
        def spatial_mask_value(row, col):
            if row % 2 == 0:
                ret = 1 if col % 2 == 0 else 0
            else:
                ret = 0 if col % 2 == 0 else 1
                
            return ret if self.mask_type == 'check_even' else 1 - ret
        
        def channel_mask(chn):
            return 1 - chn % 2 if self.mask_type == 'channel_even' else chn % 2
        
        data = np.ones(input_shape)
        for row in range(input_shape[0]):
            for col in range(input_shape[1]):
                for chn in range(input_shape[2]):
                    if self.mask_type in ['check_even', 'check_odd']:
                        data[row, col, chn] = spatial_mask_value(row, col)
                    else:
                        assert self.mask_type in ['channel_even', 'channel_odd']
                        # channel mask
                        data[row, col, chn] = channel_mask(chn)
        
        return K.constant(np.ravel(data), dtype='float32', shape=input_shape)

    def build(self, input_shape):
        assert len(input_shape) == 4, \
        'Layer assumes a (batch, row, col, chn) dimensions got {}'.format(input_shape)
        
        # Assume channel_last (tensorflow)
        channel_axis = -1
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs '
                             'should be defined. Found `None`.')
        input_dim = input_shape[channel_axis]
        
        self.mask = self.build_mask(input_shape[1:])

        # Set input spec.
        self.input_spec = InputSpec(ndim=len(input_shape), 
                                    axes={channel_axis: input_dim})
        self.built = True

    def call(self, inputs):
        return self.mask * inputs 

In [128]:
shape = (4, 4, 3)
test_tensor = np.ceil(np.random.rand(*((1,) + shape)))

def build_network(mask_type):
    x = Input(shape=shape)
    out = Mask(mask_type)(x)
    model = Model(inputs=x, outputs=out)
    model.compile(optimizer='rmsprop', loss='mse')
    
    return model

In [133]:
model = build_network('check_even')
out = model.predict(test_tensor)
assert (out[0, :, :, 0] == out[0, :, :, 1]).all()
assert (out[0, :, :, 0] == out[0, :, :, 2]).all()
out[0, :, :, 0]

array([[1., 0., 1., 0.],
       [0., 1., 0., 1.],
       [1., 0., 1., 0.],
       [0., 1., 0., 1.]], dtype=float32)

In [134]:
model = build_network('check_odd')
out = model.predict(test_tensor)
assert (out[0, :, :, 0] == out[0, :, :, 1]).all()
assert (out[0, :, :, 0] == out[0, :, :, 2]).all()
out[0, :, :, 0]

array([[0., 1., 0., 1.],
       [1., 0., 1., 0.],
       [0., 1., 0., 1.],
       [1., 0., 1., 0.]], dtype=float32)

In [137]:
model = build_network('channel_even')
out = model.predict(test_tensor)
assert (out[0, :, :, 0] == out[0, :, :, 2]).all()
assert (out[0, :, :, 1] == 0).all()
print(out[0, :, :, 0])
print(out[0, :, :, 1])

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [140]:
model = build_network('channel_odd')
out = model.predict(test_tensor)
assert (out[0, :, :, 0] == 0).all()
assert (out[0, :, :, 1] == 1).all()
assert (out[0, :, :, 0] == 0).all()
print(out[0, :, :, 0])
print(out[0, :, :, 1])

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


# 2019-07-28

* Create a simple masking layer that I can use as a building block in the network